In [3]:
import pandas as pd
import os
# os.listdir('../dataset_sheets')
df = pd.read_excel('../dataset_sheets/data-hunters.xls')

In [4]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig


In [5]:
print(df.columns)
df = df [['Title', 'Data Category']]

Index(['Permalink', 'Title', 'Content', 'Use Case', 'Data Category'], dtype='object')


In [6]:
allcats = {}
for i,row in df.iterrows():
    temp = row['Data Category'].split('|')
    # print(temp)
    for q in temp:
        if q in allcats:
            allcats[q] += 1
        else:
            allcats[q] = 1

ac = dict(sorted(allcats.items(), key=lambda item: item[1], reverse=True))
print(ac)
print(len(ac))
# ac = dict(sorted(allcats.items(), key=lambda item: item[1]))



{'Demographic Data': 4850, 'Census and Population Data': 3696, 'Economic Data': 3377, 'Construction Industry Data': 1298, 'Employment Data': 1033, 'B2B Intent Data': 485, 'Company Data': 433, 'Stock & Market Data': 427, 'Environmental Data': 412, 'Tourism Data': 355, 'Traffic and Routing Data': 343, 'Education Industry Data': 298, 'Firmographic Data': 289, 'Real Estate Data': 261, 'Trade Data': 257, 'Web Data': 208, 'Energy Data': 202, 'Geospatial Data': 199, 'Agriculture Data': 186, 'Healthcare Data': 179, 'Business Credit Rating Data': 164, 'Marketing Attribution Data': 158, 'Individual Data': 142, 'Business Registry Data': 142, 'B2B Data': 123, 'Online/Mobile Data': 111, 'SEO & Web Advertising Data': 111, 'Location Data': 110, 'Retail & Commerce Data': 108, 'Climate and Weather Data': 103, 'Legal Case Data': 95, 'Legal and IP Data': 94, 'Consumer Lifestyle Data': 93, 'Customer Transaction Data': 87, 'Industry-Specific Data': 79, 'B2B Marketing Fit Data': 71, 'Cross-Device Identity D

In [7]:
acc = {k:v for k,v in ac.items() if v > 50}
acc
len(acc)


41

In [8]:
def vectorize(s, ac):
    vector = [1 if i in s.split('|') else 0 for i in ac]
    return vector


In [9]:
df['List'] = df['Title'].apply(vectorize, ac=acc)



In [10]:
print(df.columns)
print(df['List'][:10])
df_new = df[['Title', 'List']]

Index(['Title', 'Data Category', 'List'], dtype='object')
0    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
6    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
8    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: List, dtype: object


In [11]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 100
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [12]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.Title
        self.targets = self.data.List
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [13]:
train_size = 0.8
train_dataset=df_new.sample(frac=train_size,random_state=200)
test_dataset=df_new.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df_new.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (11165, 2)
TRAIN Dataset: (8932, 2)
TEST Dataset: (2233, 2)


In [23]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [24]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 41)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [25]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [26]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [27]:
def train(epoch):
    print(len(training_loader))
    model.train()
    for _,data in enumerate(training_loader, 0):
        print(_)
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [28]:
for epoch in range(EPOCHS):
    train(epoch)

1117
